# Perform bootstrap to estimate stability of networks and network measures

In [ ]:
from inverse_covariance import (
    QuicGraphicalLasso,
    QuicGraphicalLassoCV,
    QuicGraphicalLassoEBIC,
    AdaptiveGraphicalLasso,
    ModelAverage,
)

import sys
import numpy as np
import tabulate
import time

import logging
import copy

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
import matplotlib.pyplot as plt
import os
import sys
import re
import glob
import ast
import os.path as op
import pickle
import scipy
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp

from collections import defaultdict

from copy import deepcopy
import copy

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.utils import resample
from sklearn.covariance import GraphicalLassoCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression

from itertools import combinations
from scipy.stats import ttest_ind

from scipy.stats import pearsonr
from scipy.spatial import distance
import scipy.stats as stats

import networkx as nx
import glob, os
from functools import partial

from scipy.stats import pearsonr

import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def adaptive_model_average_sklearn(
    X, 
    method, penalization='random', 
    n_trials=100, 
    metric='log_likelihood', 
    support_thresh=0.5, 
    cv=10, 
    lam=None, 
    alphas=np.logspace(-5, 1, num=20)
):
    """Run ModelAverage in default mode (QuicGraphicalLassoCV) to obtain proportion
    matrix.

    NOTE:  Only method = 'binary' really makes sense in this case.
    """
    n_trials = n_trials

    if lam is None:
        cv_model = GraphicalLassoCV(
            alphas=alphas,
            cv=cv
        )
        cv_model.fit(X)
        lam = cv_model.alpha_
    else:
        lam = lam

    model = AdaptiveGraphicalLasso(
        estimator=ModelAverage(
            n_trials=n_trials, 
            penalization=penalization, 
            lam=lam, 
            n_jobs=10, 
            support_thresh=support_thresh,
            # subsample = 0.6,
        ),
        method=method,
    )
    model.fit(X)
    lam_norm_ = np.linalg.norm(model.estimator_.lam_)
    return model.estimator_.precision_

In [ ]:
def learn_graph_structure_adaptive_average_sklearn(
    df, 
    n_trials=100, 
    penalization='random',
    score_metric="log_likelihood", 
    cv=10, 
    lam=None,
    threshold=0.5, 
    alphas=np.logspace(-5, 1, num=20)
):
    
    # standardize the time series: using correlations rather than covariance
    # former is more efficient for structure recovery
    X = df.to_numpy()
    X -= X.mean(axis=0)
    X /= X.std(axis=0)

    prec_adaptive = adaptive_model_average_sklearn(
        X, 
        penalization='random',
        method='binary', 
        n_trials=n_trials, 
        metric=score_metric,
        support_thresh=threshold,
        cv=cv,
        lam=lam,
        alphas=alphas,
    )
    
    precision_matrix_df = pd.DataFrame(prec_adaptive, columns = df.columns, index = df.columns)
            
    return precision_matrix_df

In [ ]:
def estimate_graph(
    X, 
    n_trials=1000, 
    score_metric="log_likelihood", 
    cv=3, 
    threshold=0.65, 
    alphas=np.linspace(0.01, 0.1, 20)
):
    precision_matrix_df = learn_graph_structure_adaptive_average_sklearn(
            X, 
            penalization='random',
            n_trials=n_trials,
            score_metric=score_metric,
            cv=cv,
            lam=None,
            alphas=alphas,
            threshold=threshold,
    )

    this_links = get_links(precision_matrix_df)
    this_links['weight'] = this_links['weight'].abs()
    G_ = nx.from_pandas_edgelist(this_links,'var1','var2', edge_attr='weight', create_using=nx.Graph())
    
    return G_, precision_matrix_df

In [ ]:
def get_links(precision_matrix_df, threshold=0.02):
    """
    Creates a DataFrame of links between nodes based on the provided precision matrix.

    Parameters
    ----------
    precision_matrix_df : pd.DataFrame
        A DataFrame that consists of 3 columns: var1, var2, and weight.
    threshold : float
        Threshold for inverse covariance estimate to be considered as non-zero. Allows to filter out very weak connections. Default is 0.2.

    Returns
    -------
    links : pd.DataFrame
        A DataFrame that consists of 3 columns: var1, var2, and weight.
    """
    precision_matrix_df = precision_matrix_df.where(np.triu(np.ones(precision_matrix_df.shape)).astype(bool))

    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[ (abs(links['weight']) > threshold) &  (links['var1'] != links['var2']) ]
    links = links.round(3)
    
    return links

## Read data

In [ ]:
# dataset = train | test
dataset = 'train'
test = False if dataset == 'train' else True

Load train dataset

In [ ]:
results_ern_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_models_{dataset}_id_clean.pkl")
results_ern_lat_demo_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_cov_fal_models_{dataset}_id_clean.pkl")
results_crn_df = pd.read_pickle(f"new_data/models_pickles_new_dass/crn_models_{dataset}_id_clean.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"new_data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}_id_clean.pkl")

results_ern_crn_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_crn_models_{dataset}_id_clean.pkl")
results_ern_crn_lat_demo_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_crn_cov_fal_models_{dataset}_id_clean.pkl")

train_datasets = [results_ern_df, results_ern_lat_demo_df, results_crn_df, results_crn_lat_demo_df, results_ern_crn_df, results_ern_crn_lat_demo_df]

Load test dataset

In [ ]:
results_ern_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_models_test_id_clean.pkl")
results_ern_lat_demo_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_cov_fal_models_test_id_clean.pkl")
results_crn_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/crn_models_test_id_clean.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/crn_cov_fal2_models_test_id_clean.pkl")

results_ern_crn_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_crn_models_test_id_clean.pkl")
results_ern_crn_lat_demo_test_df = pd.read_pickle(f"new_data/models_pickles_new_dass/ern_crn_cov_fal_models_test_id_clean.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())
results_ern_crn_lat_demo_test_df['e_LT_F2_C'] = results_ern_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_ern_crn_lat_demo_test_df['e_LT_F2_C'].mean())


test_datasets = [results_ern_test_df, results_ern_lat_demo_test_df, results_crn_test_df, results_crn_lat_demo_test_df, results_ern_crn_test_df, results_ern_crn_lat_demo_test_df]

In [ ]:
columns_to_drop_covs = ['WASH', 'NEU', 'Sex', 'id']
columns_to_drop = ['WASH', 'NEU',  'id']

# train dataset
ern_data_df_ = results_ern_df.drop(columns=columns_to_drop)
ern_cov_fal_data_df_ = results_ern_lat_demo_df.drop(columns=columns_to_drop_covs)

crn_data_df_ = results_crn_df.drop(columns=columns_to_drop)
crn_cov_fal2_data_df_ = results_crn_lat_demo_df.drop(columns=columns_to_drop_covs)

ern_crn_data_df_ = results_ern_crn_df.drop(columns=columns_to_drop)
ern_crn_cov_fal_data_df_ = results_ern_crn_lat_demo_df.drop(columns=columns_to_drop_covs)

# test dataset
ern_data_df_test_ = results_ern_test_df.drop(columns=columns_to_drop)
ern_cov_fal_data_df_test_ = results_ern_lat_demo_test_df.drop(columns=columns_to_drop_covs)

crn_data_df_test_ = results_crn_test_df.drop(columns=columns_to_drop)
crn_cov_fal2_data_df_test_ = results_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs)

ern_crn_data_df_test_ = results_ern_crn_test_df.drop(columns=columns_to_drop)
ern_crn_cov_fal_data_df_test_ = results_ern_crn_lat_demo_test_df.drop(columns=columns_to_drop_covs)


datasets_train = [ern_data_df_, ern_cov_fal_data_df_, crn_data_df_, crn_cov_fal2_data_df_, ern_crn_data_df_, ern_crn_cov_fal_data_df_]
datasets_test = [ern_data_df_test_, ern_cov_fal_data_df_test_, crn_data_df_test_, crn_cov_fal2_data_df_test_, ern_crn_data_df_test_, ern_crn_cov_fal_data_df_test_]

## Perform full bootstrap

In [ ]:
def bootstrap_network(
    X, 
    model='adaptive_sklearn', 
    N=100, 
):
    bootstrapped_matrices = []
    n = len(X)
    for i in range(N):
        
        # Generate a bootstrap sample
        bootstrap_X = resample(X, n_samples=n, replace=True)
        print(f'{i} iteration')
        _, precision_matrix = estimate_graph(bootstrap_X)
        bootstrapped_matrices.append(precision_matrix)
            
    return bootstrapped_matrices

Perform bootstrapping

In [ ]:
logging.getLogger().setLevel(logging.INFO)

for index, dataset in enumerate(datasets_train):
    print(f'Estimating {index} dataset ######################')

    bootstraped_precision_matrices = bootstrap_network(
        X = dataset,
        N=1000,
    )

    with open(f'new_data/bootstrap_results/bootstrap_precision_matrices_{index}.pkl', 'wb') as f:
        pickle.dump(bootstraped_precision_matrices, f)

## Estimate the stability of network measures using bootstrapping

In [ ]:
def calculate_nodes_predictability(X, G):
    explained_variance = dict()
        
    for node in G.nodes():
        y_ = X[[node]]

        neighbors = list(G.neighbors(node))

        X_ = X.loc[:, neighbors]

        lm = LinearRegression()
        lm.fit(X_, y_)

        score = lm.score(X_,y_)
        explained_variance[node] = score

    return explained_variance

In [ ]:
def get_links(precision_matrix_df, threshold=0.02):
    precision_matrix_df = precision_matrix_df.where(np.triu(np.ones(precision_matrix_df.shape)).astype(bool))
    
    links = precision_matrix_df.stack().reset_index()
    links.columns = ['var1', 'var2','weight']
    links=links.loc[ (abs(links['weight']) > threshold) &  (links['var1'] != links['var2']) ]
        
    links = links.round(3)
    
    return links

In [ ]:
def get_ranked_dict(dict_):
    items = [(key, value) for key, value in dict_.items()]
    sorted_items = sorted(items, key=lambda x: x[1], reverse=True)

    ranked_dict = {}
    rank = 1

    for key, value in sorted_items:
        ranked_dict[key] = rank
        rank += 1

    return ranked_dict

In [ ]:
def bootstrap_network_measures(
    X, 
    model='adaptive_sklearn', 
    measures=None, 
    N=1000, 
    levels=np.arange(.95, 0.25, -0.05)
):
    results_df = pd.DataFrame()
    network_measures_baseline = []
    
    G_, _ = estimate_graph(X)
    
    for measure, measure_parameters in measures: 
        network_measure = measure(G = G_, **measure_parameters)
        ranked_network_measure = get_ranked_dict(network_measure)
        ranked_network_measure = {k: v for k, v in sorted(ranked_network_measure.items(), key=lambda item: item[0], reverse=True)}
        logging.info(ranked_network_measure)
        baseline_measure = list(ranked_network_measure.values())
        
        network_measures_baseline.append(baseline_measure)
        logging.info('Baseline measures appended')
    
    for level in levels:
        n = int(level * len(X))
        print(f'X len before samples: {n}')

        for i in range(N):
            # Generate a bootstrap sample with replacements
            bootstrap_X = resample(X, n_samples=n, replace=True)
            print(f'X len inside samples: {len(bootstrap_X)}')
            G_, _ = estimate_graph(bootstrap_X)
            
            for index, network_measure in enumerate(measures): 
                measure, measure_parameters = network_measure
                
                current_baseline = network_measures_baseline[index]
                
                try:
                    network_measure = measure(G = G_, **measure_parameters)
                    ranked_network_measure = get_ranked_dict(network_measure)
                    ranked_network_measure = {k: v for k, v in sorted(ranked_network_measure.items(), key=lambda item: item[0], reverse=True)}
                    logging.info(ranked_network_measure)
                    ranked_network_measure = list(ranked_network_measure.values())

                    try:
                        similarity_corr_coef, p_value = scipy.stats.pearsonr(current_baseline, ranked_network_measure)
                        print(f"Measure: {measure.__name__}  Level: {level}   sample: {i}   : similarity: {similarity_corr_coef}")
                        this_results = pd.DataFrame({
                            'measure': [measure.__name__],
                            'level': [level],
                            'similatity': [similarity_corr_coef],
                        })

                        results_df = pd.concat([results_df, this_results], ignore_index = True)
                    except:
                        logging.info('DIFFERENT LENGTHS OF BASELINE AND CURRENT')
                except:
                    logging.info('NETWORK MEASURE ERROR')
                    
            
    return results_df

Estimate the stability of network measures

In [ ]:
logging.getLogger().setLevel(logging.INFO)

for index, (dataset_train, dataset_test) in enumerate(zip(datasets, test_datasets)):

    df_ranked = bootstrap_network_measures(
        X = dataset_train,
        measures = [
            (calculate_nodes_predictability, {'X': dataset_test}), 
            (nx.degree_centrality, {}),
            (nx.closeness_centrality, {}),
            (nx.current_flow_closeness_centrality, {'weight': 'weight'}),
            (nx.betweenness_centrality, {'weight': 'weight'}),
            (nx.current_flow_betweenness_centrality, {'weight': 'weight'}),
            (nx.load_centrality, {})
        ],
        N=2,
        levels=np.arange(.95, 0.25, -0.05)

    )
    # df_ranked.to_pickle(f'data/network_analysis/stability_estimates/network_measures_ranked_bootstrapped_{index}_pred_test.pkl')